## Import Packages

In [ ]:
pip install keras-rectified-adam

In [29]:
# import useful packages
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cv2
import os
import dill
import glob
#from keras_radam import RAdam
from random import shuffle
from google.colab import drive
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, f1_score, accuracy_score, confusion_matrix
from keras import layers, models, optimizers
from keras.layers import GlobalAveragePooling2D,Dense,Dropout,Input,Conv2D,UpSampling2D,MaxPool2D,Flatten
from imgaug import augmenters as iaa
from keras.callbacks import ModelCheckpoint, Callback
from math import ceil
from keras.layers import GlobalAveragePooling2D,Dense,Dropout
from keras.models import Model,load_model
from keras.optimizers import Adam,SGD
from keras.losses import huber_loss

Biuld connections with Google drive for data import

In [2]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Image Generation & Augmentations

### Keras biult-in

Data Import

In [4]:
# import the file of labels
labels = pd.read_csv('/content/gdrive/My Drive/aptos2019-blindness-detection/train.csv')
# shuffle the data set
labels = labels.sample(frac=1).reset_index(drop=True)

In [5]:
# add file type to the column of file names
labels['id_code'] = labels['id_code'].astype(str) + '.png'

# encode the label column to a categorical variable
labels['diagnosis'] = labels['diagnosis'].astype(str)

In [6]:
# randomly split the labeled images into a training set and a test set in a ratio of 8:2
train_and_val, test = train_test_split(labels, test_size=0.2,random_state=1002965829)
# randomly subset 1/10 of the training set to form a validation set
train, val = train_test_split(train_and_val, test_size=0.1,random_state=1002965829)

Generate image batches for the model

In [7]:
# define a training image generator
#train_gen = ImageDataGenerator(rescale = 1./255, 
                               #rotation_range = 10,
                               #shear_range = 0.2, 
                               #zoom_range = 0.2,
                               #horizontal_flip = True)

train_gen = ImageDataGenerator(rescale = 1./255)

# define a test image generator
test_gen = ImageDataGenerator(rescale=1./255)

# define a validation image generator
val_gen = ImageDataGenerator(rescale=1./255)

In [8]:
# define the image file path
image_dir = '/content/gdrive/My Drive/aptos2019-blindness-detection/train_images'

# generate training images
train_images = train_gen.flow_from_dataframe(
    dataframe = train, 
    directory = image_dir, 
    x_col='id_code', 
    y_col='diagnosis',
    target_size=(350, 350),
    batch_size=128, 
    class_mode='categorical')

Found 2636 validated image filenames belonging to 5 classes.


In [9]:
# generate test images
test_images = test_gen.flow_from_dataframe(
    dataframe = test, 
    directory = image_dir, 
    x_col='id_code', 
    y_col='diagnosis',
    target_size=(350, 350),
    batch_size = 64,
    class_mode='categorical')

Found 733 validated image filenames belonging to 5 classes.


In [10]:
# generate validation images
val_images = val_gen.flow_from_dataframe(
    dataframe = val, 
    directory = image_dir, 
    x_col='id_code', 
    y_col='diagnosis',
    target_size=(350, 350),
    batch_size = 64,
    class_mode='categorical')

Found 293 validated image filenames belonging to 5 classes.


In [ ]:
# inspect the processed training data
for batch, label in train_images:
  for i in range(4):
    plt.subplot(2,2,i+1)
    plt.imshow(batch[i])
  break

In [ ]:
for batch in train_images[0]:
  for i in range(10):
    print(batch[i].shape)
  break

### Customized

Data Import

In [3]:
# import the file of labels
labels = pd.read_csv('/content/gdrive/My Drive/aptos2019-blindness-detection/train.csv')

# add file type to the column of file names
labels['id_code'] = labels['id_code'].astype(str) + '.png'

# encode the label column to a categorical variable
labels['diagnosis'] = labels['diagnosis'].astype(str)

# randomly split the labeled images into a training set and a test set in a ratio of 8:2
train_val, test = train_test_split(labels, test_size=0.2, random_state=1002965829)

# randomly subset 1/10 of the training set to form a validation set
train, val = train_test_split(train_val, test_size=0.1, random_state=1002965829)

Define a Customized Image Generator

In [6]:
class data_generator(object):
  
  #####------------------------ Image Augmentation ----------------------#####
  def augment(self, img, img_type, img_size):
    # img: a single image
    # image_type: train or val or test
    # image_size: resized image size
    #----------------------------------------------------------------------#
    def clip_black_out(img):
      m,n,k=img.shape
      left_count=0
      for j in range(n):
        if (img[:,j]<=10).all():
          left_count=left_count+1
        else:
          break
      right_count=n
      for j in range(n-1,0,-1):
        if (img[:,j]<=10).all():
          right_count=right_count-1
        else:
          break
      upper_count=0
      for j in range(m):
        if (img[j,:]<=10).all():
          upper_count=upper_count+1
        else:
          break
      lower_count=m
      for j in range(m-1,0,-1):
        if (img[j,:]<=10).all():
          lower_count=lower_count-1
        else:
          break
      img=img[upper_count:lower_count,left_count:right_count]
      return img
    #----------------------------------------------------------------------#
    def crop_image_from_gray(img,tol=7):
      if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
      elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
          return img # return original image
        else:
          img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
          img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
          img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
          img = np.stack([img1,img2,img3],axis=-1)
        return img

    def load_ben_color(img, sigmaX=40):
      img = crop_image_from_gray(img)
      img = cv2.addWeighted (img, 4, cv2.GaussianBlur(img, (0,0), sigmaX), -4, 128)
      return img
    #----------------------------------------------------------------------#
    if img_type == 'validation' or 'test':
      img = load_ben_color(clip_black_out(img))
      img = cv2.resize(img,(img_size, img_size))
      return img
    if img_type == 'training':
      augmenter = iaa.Sequential(
          [iaa.OneOf([iaa.Affine(rotate=0), iaa.Affine(rotate=5), iaa.Affine(rotate=10)]),
           iaa.OneOf([iaa.Fliplr(0.5)])], 
           random_order=True)
      img = augmenter.augment_image(load_ben_color(clip_black_out(img)))
      img = cv2.resize(img,(img_size,img_size))
      return img

  #####------------------------ Image Import ----------------------#####
  def load_image(self, path, img_type, img_size, augment=False):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    if augment:
      return self.augment(img, img_type, img_size)/255.
    else:
      img=cv2.resize(img,(img_size, img_size))
      return img/255.
  
  ######------------------------ Image Batch Mix-up ----------------------#####
  def mix_up(self, x_batch, y_batch):
    x_batch= np.array(x_batch, np.float32)
    y_batch=np.array(y_batch, np.float32)
    lam = np.random.beta(0.2, 0.4)
    ori_index = np.arange(int(len(x_batch))).astype('int')
    index_array = np.arange(int(len(x_batch))).astype('int')
    np.random.shuffle(index_array) 
     
    mixed_x = lam * x_batch[ori_index] + (1 - lam) * x_batch[index_array]
    mixed_y = lam * y_batch[ori_index] + (1 - lam) * y_batch[index_array]
    return mixed_x, mixed_y
  
  ######--------------------------- Batch Generator -------------------------#####
  def create_batch(self, my_dir, pred_mod, data_info, data_type, batch_size, resize_to, 
                   augment=False, mix=False):
    # my_dir: the base directory to the image files
    # pred_mod: treat the labels as classfification or regression
    # data_info: paths to each image and their labels
    # data_type: train, val, or test
    # batch_size: batch_size
    # resize_to: rescaled size
    # augment: whether do image augmentations 
    # mix: whether mix up each batch
    assert pred_mod in ['cls', 'reg'], 'incorrect prediction mode inputted'
    assert data_type in ['training', 'validation', 'test'], 'incorrect data type inputted'

    while True:
      if data_type == 'training':
        data_info = data_info.sample(frac=1).reset_index(drop=True)
      for start in range(0, len(data_info), batch_size):
        end = min(start + batch_size, len(data_info))
        batch_info = data_info.iloc[start:end]
        batch_images=[]
        batch_labels = []
        for i in range(len(batch_info)):
          image = self.load_image(os.path.join(my_dir,batch_info.iloc[i]['id_code']), 
              data_type, 
              resize_to, 
              augment)
          batch_images.append(image)
          if pred_mod == 'cls':
            num_classes = len(pd.unique(data_info.iloc[:,1]))
            one_hot = np.zeros(num_classes)
            one_hot[int(batch_info.iloc[i,1:].values)] = 1
            batch_labels.append(one_hot)
          else:
            batch_labels.append(np.array(batch_info.iloc[i,1:].values, np.float32)/4.)
        if mix:
          batch_images, batch_labels = self.mix_up(batch_images, batch_labels)
        yield np.array(batch_images), np.array(batch_labels, np.float32)

In [7]:
# define an image generator
image_gen = data_generator()

# generate training data
train_images = image_gen.create_batch(
    my_dir = '/content/gdrive/My Drive/aptos2019-blindness-detection/train_images', 
    pred_mod = 'cls',
    data_info = train,
    data_type = 'training',
    batch_size = 128,
    resize_to = 350,
    augment = False,
    mix = False)

# generate validation data
val_images = image_gen.create_batch(
    my_dir = '/content/gdrive/My Drive/aptos2019-blindness-detection/train_images', 
    pred_mod = 'cls',
    data_info = val,
    data_type = 'validation',
    batch_size = 64,
    resize_to = 350,
    augment = False,
    mix = False)

# generate test data
test_images = image_gen.create_batch(
    my_dir = '/content/gdrive/My Drive/aptos2019-blindness-detection/train_images', 
    pred_mod = 'cls',
    data_info = test,
    data_type = 'test',
    batch_size = 64,
    resize_to = 350,
    augment = False,
    mix = False)

In [ ]:
# inspect the image processing
for batch, label in train_images:
  for i in range(4):
    plt.subplot(2,2,i+1)
    plt.imshow(batch[i])
  break

## Training Techniques

### Focal loss function

Binary Focal Loss

In [8]:
def binary_focal_loss(gamma=2., alpha=.25):
    """
    Binary form of focal loss.
      FL(p_t) = -alpha * (1 - p_t)**gamma * log(p_t)
      where p = sigmoid(x), p_t = p or 1 - p depending on if the label is 1 or 0, respectively.
    References:
        https://arxiv.org/pdf/1708.02002.pdf
    Usage:
     model.compile(loss=[binary_focal_loss(alpha=.25, gamma=2)], metrics=["accuracy"], optimizer=adam)
    """

    def binary_focal_loss_fixed(y_true, y_pred):
        """
        :param y_true: A tensor of the same shape as `y_pred`
        :param y_pred:  A tensor resulting from a sigmoid
        :return: Output tensor.
        """
        y_true = tf.cast(y_true, tf.float32)
        # Define epsilon so that the back-propagation will not result in NaN for 0 divisor case
        epsilon = K.epsilon()
        # Add the epsilon to prediction value
        # y_pred = y_pred + epsilon
        # Clip the prediciton value
        y_pred = K.clip(y_pred, epsilon, 1.0 - epsilon)
        # Calculate p_t
        p_t = tf.where(K.equal(y_true, 1), y_pred, 1 - y_pred)
        # Calculate alpha_t
        alpha_factor = K.ones_like(y_true) * alpha
        alpha_t = tf.where(K.equal(y_true, 1), alpha_factor, 1 - alpha_factor)
        # Calculate cross entropy
        cross_entropy = -K.log(p_t)
        weight = alpha_t * K.pow((1 - p_t), gamma)
        # Calculate focal loss
        loss = weight * cross_entropy
        # Sum the losses in mini_batch
        loss = K.mean(K.sum(loss, axis=1))
        return loss

    return binary_focal_loss_fixed

Multi-class Focal Loss

In [9]:
def categorical_focal_loss(alpha=[[0.25, 0.25, 0.25, 0.25, 0.25]], gamma=2.):
    """
    Softmax version of focal loss.
    When there is a skew between different categories/labels in your data set, you can try to apply this function as a
    loss.
           m
      FL = ∑  -alpha * (1 - p_o,c)^gamma * y_o,c * log(p_o,c)
          c=1
      where m = number of classes, c = class and o = observation
    Parameters:
      alpha -- the same as weighing factor in balanced cross entropy. Alpha is used to specify the weight of different
      categories/labels, the size of the array needs to be consistent with the number of classes.
      gamma -- focusing parameter for modulating factor (1-p)
    Default value:
      gamma -- 2.0 as mentioned in the paper
      alpha -- 0.25 as mentioned in the paper
    References:
        Official paper: https://arxiv.org/pdf/1708.02002.pdf
        https://www.tensorflow.org/api_docs/python/tf/keras/backend/categorical_crossentropy
    Usage:
     model.compile(loss=[categorical_focal_loss(alpha=[[.25, .25, .25]], gamma=2)], metrics=["accuracy"], optimizer=adam)
    """

    alpha = np.array(alpha, dtype=np.float32)

    def categorical_focal_loss_fixed(y_true, y_pred):
        """
        :param y_true: A tensor of the same shape as `y_pred`
        :param y_pred: A tensor resulting from a softmax
        :return: Output tensor.
        """

        # Clip the prediction value to prevent NaN's and Inf's
        epsilon = K.epsilon()
        y_pred = K.clip(y_pred, epsilon, 1. - epsilon)

        # Calculate Cross Entropy
        cross_entropy = -y_true * K.log(y_pred)

        # Calculate Focal Loss
        loss = alpha * K.pow(1 - y_pred, gamma) * cross_entropy

        # Compute mean loss in mini_batch
        return K.mean(K.sum(loss, axis=-1))

    return categorical_focal_loss_fixed

### Huber loss function

In [13]:
def smooth_L1_loss(y_true, y_pred):
    return huber_loss(y_true, y_pred)

### Cosine annealing

In [10]:
def get_1cycle_schedule(lr_max=1e-3, n_data_points=8000, epochs=200, batch_size=40, verbose=0):          
    """
    Creates a look-up table of learning rates for 1cycle schedule with cosine annealing
    See @sgugger's & @jeremyhoward's code in fastai library: https://github.com/fastai/fastai/blob/master/fastai/train.py
    Wrote this to use with my Keras and (non-fastai-)PyTorch codes.
    Note that in Keras, the LearningRateScheduler callback (https://keras.io/callbacks/#learningratescheduler) only operates once per epoch, not per batch
      So see below for Keras callback

    Keyword arguments:
    lr_max            chosen by user after lr_finder
    n_data_points     data points per epoch (e.g. size of training set)
    epochs            number of epochs
    batch_size        batch size
    Output:  
    lrs               look-up table of LR's, with length equal to total # of iterations
    Then you can use this in your PyTorch code by counting iteration number and setting
          optimizer.param_groups[0]['lr'] = lrs[iter_count]
    """
    if verbose > 0:
        print("Setting up 1Cycle LR schedule...")
    pct_start, div_factor = 0.3, 25.        # @sgugger's parameters in fastai code
    lr_start = lr_max/div_factor
    lr_end = lr_start/1e4
    n_iter = ((n_data_points -1)* epochs // batch_size) + 1    # number of iterations
    a1 = int(n_iter * pct_start)
    a2 = n_iter - a1

    # make look-up table
    lrs_first = np.linspace(lr_start, lr_max, a1)            # linear growth
    lrs_second = (lr_max-lr_end)*(1+np.cos(np.linspace(0,np.pi,a2)))/2 + lr_end  # cosine annealing
    lrs = np.concatenate((lrs_first, lrs_second))
    return lrs


class OneCycleScheduler(Callback):
    """My modification of Keras' Learning rate scheduler to do 1Cycle learning
       which increments per BATCH, not per epoch
    Keyword arguments
        **kwargs:  keyword arguments to pass to get_1cycle_schedule()
        Also, verbose: int. 0: quiet, 1: update messages.

    Sample usage (from my train.py):
        lrsched = OneCycleScheduler(lr_max=1e-4, n_data_points=X_train.shape[0],
        epochs=epochs, batch_size=batch_size, verbose=1)
    """
    def __init__(self, **kwargs):
        super(OneCycleScheduler, self).__init__()
        self.verbose = kwargs.get('verbose', 0)
        self.lrs = get_1cycle_schedule(**kwargs)
        self.iteration = 0

    def on_batch_begin(self, batch, logs=None):
        lr = self.lrs[self.iteration]
        K.set_value(self.model.optimizer.lr, lr)         # here's where the assignment takes place
        '''
        if self.verbose > 3:
            print('\nIteration %06d: OneCycleScheduler setting learning '
                  'rate to %s.' % (self.iteration, lr))
        '''
        self.iteration += 1

    def on_epoch_end(self, epoch, logs=None):  # this is unchanged from Keras LearningRateScheduler
        logs = logs or {}
        logs['lr'] = K.get_value(self.model.optimizer.lr)
        # self.iteration = 0

## CNN

### V1 for classificastion

In [ ]:
# construct a CNN
model_v1 = models.Sequential()

model_v1.add(layers.Conv2D(128, (5, 5), activation = 'relu',
                                 input_shape = (350, 350, 3))) 
model_v1.add(layers.MaxPooling2D((2, 2))) 
model_v1.add(layers.Conv2D(64, (3, 3), activation = 'relu')) 
model_v1.add(layers.MaxPooling2D((2, 2)))
model_v1.add(layers.Conv2D(64, (3, 3), activation = 'relu')) 
model_v1.add(layers.MaxPooling2D((2, 2)))
model_v1.add(layers.Conv2D(64, (3, 3), activation = 'relu')) 
model_v1.add(layers.MaxPooling2D((2, 2)))
model_v1.add(layers.Conv2D(64, (3, 3), activation = 'relu'))
model_v1.add(layers.MaxPooling2D((2, 2))) 
model_v1.add(layers.Conv2D(64, (3, 3), activation = 'relu'))
model_v1.add(layers.Conv2D(64, (3, 3), activation = 'relu'))
model_v1.add(layers.Conv2D(64, (3, 3), activation = 'relu'))  
model_v1.add(layers.Flatten())
model_v1.add(layers.Dense(128, activation = 'relu'))
model_v1.add(layers.Dense(64, activation = 'relu'))
model_v1.add(layers.Dense(5, activation = 'softmax'))

model_v1.summary()

In [25]:
model_v1.compile(loss = [categorical_focal_loss()],
              optimizer = 'adam', 
              metrics=['acc'])

In [ ]:
num_epochs = 8
train_steps = ceil(len(train)/128)
val_steps = ceil(len(val)/64)

# define the check point
filepath = '/content/gdrive/My Drive/aptos2019-blindness-detection/model_v1'
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc', verbose = 1, 
                             save_best_only = True, mode = 'max', save_weights_only = False)

# define the cosine leraning rate
lrsched = OneCycleScheduler(lr_max = 0.001, n_data_points = len(train),
        epochs = num_epochs, batch_size = 128, verbose = 0)

# train the model
model_v1.fit_generator(
    train_images,
    steps_per_epoch = train_steps,
    validation_data = val_images,
    validation_steps = val_steps,
    epochs = num_epochs,
    verbose = 1,
    callbacks = [checkpoint, lrsched])

In [ ]:
# load the model back
model_v1 = models.load_model('/content/gdrive/My Drive/aptos2019-blindness-detection/model_v1', compile = False)
model_v1.summary() 

In [28]:
# generate the network's outputs for the test set
test_steps = ceil(len(test)/64)
predictions = model_v1.predict_generator(test_images, steps = test_steps)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1976: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


In [29]:
# convert the outputs to certain categories
pred_labels = predictions.argmax(axis = 1).astype(str)
true_labels = test['diagnosis'].to_numpy()

# compute performance metrics
weighted_f1 = round(f1_score(true_labels, pred_labels, average = 'weighted'), 4) 
accuracy = round(accuracy_score(true_labels, pred_labels), 4) 
kappa = round(cohen_kappa_score(true_labels, pred_labels), 4)

# display the metrics
print("Accuracy: {}".format(accuracy) + '\n'
      + "Weigjted F1: {}".format(weighted_f1) + '\n'
      + "Cohen's Kappa: {}".format(kappa))

# display the confusion matrix
confusion_matrix(true_labels, pred_labels)

Accuracy: 0.3752
Weigjted F1: 0.3499
Cohen's Kappa: -0.0027


array([[191,  32, 134,   5,   0],
       [ 38,   4,  25,   2,   0],
       [112,  12,  78,   0,   0],
       [ 18,   5,   9,   2,   0],
       [ 37,   3,  26,   0,   0]])

### V2 for regression

In [ ]:
model_v2 = models.Sequential()

model_v2.add(layers.Conv2D(128, (5, 5), activation = 'relu',
                                 input_shape = (350, 350, 3))) # output shape: (254, 254, 16)
model_v2.add(layers.MaxPooling2D((2, 2))) # output shape: (127, 127, 16)
model_v2.add(layers.Conv2D(64, (3, 3), activation = 'relu')) # output shape: (125, 125, 32)
model_v2.add(layers.MaxPooling2D((2, 2))) # output shape: (62, 62, 32)
model_v2.add(layers.Conv2D(64, (3, 3), activation = 'relu')) # output shape: (60, 60, 64)
model_v2.add(layers.MaxPooling2D((2, 2))) # output shape: (30, 30, 64)
model_v2.add(layers.Conv2D(64, (3, 3), activation = 'relu')) # output shape: (60, 60, 64)
model_v2.add(layers.MaxPooling2D((2, 2))) # output shape: (30, 30, 64)
model_v2.add(layers.Conv2D(64, (3, 3), activation = 'relu')) # output shape: (60, 60, 64)
model_v2.add(layers.MaxPooling2D((2, 2)))
model_v2.add(layers.Conv2D(64, (3, 3), activation = 'relu')) # output shape: (60, 60, 64)
model_v2.add(layers.Flatten())
model_v2.add(layers.Dense(64, activation = 'relu'))
model_v2.add(layers.Dense(32, activation = 'relu'))
model_v2.add(layers.Dense(1, activation = 'linear'))
model_v2.summary()

In [14]:
model_v2.compile(loss = [smooth_L1_loss],
              optimizer = 'adam', 
              metrics=['mae'])

In [ ]:
num_epochs = 10
train_steps = ceil(len(train)/128)
val_steps = ceil(len(val)/64)

# define the check point
filepath = '/content/gdrive/My Drive/aptos2019-blindness-detection/model_v2'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='auto', save_weights_only = False)

# define the cosine leraning rate
lrsched = OneCycleScheduler(lr_max = 1e-3, n_data_points = len(train),
        epochs = num_epochs, batch_size = 128, verbose = 0)

# train the model
model_v2.fit_generator(
    train_images,
    steps_per_epoch = train_steps,
    validation_data = val_images,
    validation_steps = val_steps,
    epochs = num_epochs,
    verbose=1,
    callbacks = [checkpoint, lrsched])

In [ ]:
# load the model back
model_v2 = models.load_model('/content/gdrive/My Drive/aptos2019-blindness-detection/model_v2', compile = False)
model_v2.summary()

In [13]:
# generate predicted labels for the test set
test_steps = ceil(len(test)/64)
predictions = model_v2.predict_generator(test_images,steps = test_steps)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1976: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


In [30]:
pred = predictions * 4

# map the numeric results to the corresponding classes
for i in range(pred.shape[0]):
    if(pred[i]<=0.5):
        pred[i]=0
    elif(pred[i]>0.5 and pred[i]<=1.2):
        pred[i]=1
    elif(pred[i]>1.2 and pred[i]<=2.5):
        pred[i]=2
    elif(pred[i]>2.5 and pred[i]<=3.5):
        pred[i]=3
    else:
        pred[i]=4

pred_labels = pred
true_labels = test['diagnosis'].to_numpy().astype('int')

# compute performance metrics
weighted_f1 = round(f1_score(true_labels, pred_labels, average = 'weighted'), 4) # 0.3139
accuracy = round(accuracy_score(true_labels, pred_labels), 4) # 0.3138
kappa = round(cohen_kappa_score(true_labels, pred_labels), 4) # 0.0073

# display the metrics
print("Accuracy: {}".format(accuracy) + '\n'
      + "Weigjted F1: {}".format(weighted_f1) + '\n'
      + "Cohen's Kappa: {}".format(kappa))

# display the confusion matrix
confusion_matrix(true_labels, pred_labels)

Accuracy: 0.6467
Weigjted F1: 0.619
Cohen's Kappa: 0.4594


array([[297,  49,  16,   0,   0],
       [  7,   2,  60,   0,   0],
       [  4,  20, 174,   4,   0],
       [  0,   2,  31,   1,   0],
       [  3,   6,  55,   2,   0]])

## Auto-Encoder Pretrained CNN

In [17]:
# import the file of labels
labels = pd.read_csv('/content/gdrive/My Drive/aptos2019-blindness-detection/train.csv')

# add file type to the column of file names
labels['id_code'] = labels['id_code'].astype(str) + '.png'

# encode the label column to a categorical variable
labels['diagnosis'] = labels['diagnosis'].astype(str)

# randomly split the labeled images into a training set and a test set in a ratio of 8:2
train_val, test = train_test_split(labels, test_size=0.2, random_state=1002965829)

# randomly subset 1/10 of the training set to form a validation set
train, val = train_test_split(train_val, test_size=0.1, random_state=1002965829)

Combine the labeled data and unlabeled data into one dataframe


In [ ]:
# manipulate the labeled data info
train['filename']='/content/gdrive/My Drive/aptos2019-blindness-detection/train_images'
train.drop(columns=['diagnosis'],inplace=True)

# create a dataframe to contain the file paths of the unlabeled images
test_paths=glob.glob('/content/gdrive/My Drive/aptos2019-blindness-detection/test_images/*png')
def transfrom_name(file_paths):
    return os.path.split(file_paths)[1]
def transfrom_file_name(file_paths):
    return os.path.split(file_paths)[0]
train_1 = pd.DataFrame({'id_code':test_paths,'filename':test_paths})
train_1['filename']=train_1['filename'].map(transfrom_file_name)
train_1['id_code']=train_1['id_code'].map(transfrom_name)

# shuffle the combined data
all_df = pd.concat([train,train_1])
all_df = all_df.sample(frac=1).reset_index(drop=True)

Define a simple image generator for the auto-encoder model

In [26]:
class my_gen(object):
    def __init__(self, df, batchSize,image_size):
        self.df=df
        self.batchSize=batchSize
        self.numImage=self.df.shape[0]
        self.image_size=image_size
        
    def generator(self, passes=np.inf):
        epochs = 0
        while epochs <passes:
            for i in range(0, self.numImage, self.batchSize):
                paths_name=self.df.iloc[i:i+self.batchSize]['id_code'].tolist()
                paths_filename=self.df.iloc[i:i+self.batchSize]['filename'].tolist()
                
                data=np.zeros((len(paths_name),self.image_size,self.image_size,3))
                for j,name in enumerate(paths_name):
                    img=cv2.imread(os.path.join(paths_filename[j],name))
                    img=cv2.resize(img,(self.image_size,self.image_size))
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    img=self.load_ben_color(img)
                    img=img/255.0
                    data[j]=img
                yield (data,data)
        epochs=epochs+1

    def load_ben_color(self,image, sigmaX=20):
        image=cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0) , sigmaX) ,-4 ,128)

        return image

Autoencoder

In [31]:
batch_size=128
image_size=384
train_gen=my_gen(all_df,batch_size,image_size)

In [32]:
# define the architecture of the encoder/decoder model
inputs=Input(shape=(image_size,image_size,3))
x=Conv2D(filters=128,kernel_size=5,padding='same',strides=1,activation='relu')(inputs)
x=MaxPool2D()(x)
x=Conv2D(filters=64,kernel_size=5,padding='same',strides=1,activation='relu')(x)
x=MaxPool2D()(x)
x=Conv2D(filters=64,kernel_size=5,padding='same',strides=1,activation='relu')(x)
x=MaxPool2D()(x)
x=Conv2D(filters=32,kernel_size=5,padding='same',strides=1,activation='relu')(x)
encoder_out=MaxPool2D()(x)

x=UpSampling2D()(encoder_out)
x=Conv2D(filters=64,kernel_size=5,padding='same',strides=1,activation='relu')(x)
x=UpSampling2D()(x)
x=Conv2D(filters=64,kernel_size=5,padding='same',strides=1,activation='relu')(x)
x=UpSampling2D()(x)
x=Conv2D(filters=128,kernel_size=5,padding='same',strides=1,activation='relu')(x)
x=UpSampling2D()(x)
decoder_out=Conv2D(filters=3,kernel_size=5,padding='same',strides=1,activation='linear')(x)

In [33]:
encoder_model=Model(inputs,encoder_out)
auto_model=Model(inputs,decoder_out)

In [ ]:
auto_model.compile(loss='mse',optimizer='adam')
auto_model.summary()

In [ ]:
auto_model.fit_generator(train_gen.generator(),steps_per_epoch=np.ceil(all_df.shape[0]/batch_size),epochs=10)
auto_model.save('/content/gdrive/My Drive/aptos2019-blindness-detection/auto_model')
encoder_model.save('/content/gdrive/My Drive/aptos2019-blindness-detection/encoder_model')

In [ ]:
encoder_model=load_model('/content/gdrive/My Drive/aptos2019-blindness-detection/encoder_model')

### CNN V3 (pretrained + classification)

In [ ]:
# extract the trained encoder layers
x_model = encoder_model.output
x_model = Flatten()(x_model)
# connect the encoder with a fully-connected NN for classification
x_model = Dense(64, activation='relu')(x_model)
x_model = Dense(32, activation='relu')(x_model)
predictions = Dense(5, activation='softmax', name='output_layer')(x_model)
model_v3 = Model(inputs=encoder_model.input, outputs=predictions)

model_v3.summary()

In [ ]:
epochs=10
batch_size=128

filepath = '/content/gdrive/My Drive/aptos2019-blindness-detection/model_v3'
check = ModelCheckpoint(filepath=filepath,monitor='val_mae',save_best_only=True,save_weights_only=False,mode='min')

lrsched = OneCycleScheduler(lr_max=1e-3, n_data_points=train.shape[0],
        epochs=epochs, batch_size=batch_size, verbose=0)

model_v3.compile(loss=smooth_L1_loss,
              optimizer=Adam(lr=1e-3,decay=3e-4), metrics=['mae'])
model_v3.fit_generator(train_images,steps_per_epoch=ceil(train.shape[0]/batch_size),
                    validation_data=val_images,validation_steps=ceil(val.shape[0]/32),
                    epochs=epochs,callbacks=[check,lrsched])

In [ ]:
# load the model back
model_v3 = load_model('/content/gdrive/My Drive/aptos2019-blindness-detection/model_v3',compile=False)
model_v3.summary()

In [ ]:
# generate the predictions on the test set
predictions = model_v3.predict_generator(test_images,steps=np.ceil(test.shape[0]/32))

In [ ]:
# convert the outputs to certain categories
pred_labels = predictions.argmax(axis = 1).astype(str)
true_labels = test['diagnosis'].to_numpy()

# compute performance metrics
weighted_f1 = round(f1_score(true_labels, pred_labels, average = 'weighted'), 4)
accuracy = round(accuracy_score(true_labels, pred_labels), 4)
kappa = round(cohen_kappa_score(true_labels, pred_labels), 4)

# display the metrics
print("Accuracy: {}".format(accuracy) + '\n'
      + "Weigjted F1: {}".format(weighted_f1) + '\n'
      + "Cohen's Kappa: {}".format(kappa))

# display the confusion matrix
confusion_matrix(true_labels, pred_labels)

### CNN V4 (pretrained + regression)

In [ ]:
# extract the trained encoder layers
x_model = encoder_model.output
x_model = Flatten()(x_model)
# connect the encoder with a fully-connected NN for classification
x_model = Dense(64, activation='relu')(x_model)
x_model = Dense(64, activation='relu')(x_model)
predictions = Dense(1, activation='linear', name='output_layer')(x_model)
model_v4 = Model(inputs=encoder_model.input, outputs=predictions)

model_v4.summary()

In [ ]:
# train the regression model
epochs=10
batch_size=128

filepath = '/content/gdrive/My Drive/aptos2019-blindness-detection/model_v4'
check = ModelCheckpoint(filepath=filepath,monitor='val_mae',save_best_only=True,save_weights_only=False,mode='min')

lrsched = OneCycleScheduler(lr_max=1e-3, n_data_points=train.shape[0],
        epochs=epochs, batch_size=batch_size, verbose=0)

model_v4.compile(loss=smooth_L1_loss,
              optimizer=Adam(lr=1e-3,decay=3e-4), metrics=['mae'])
model_v4.fit_generator(train_images,steps_per_epoch=ceil(train.shape[0]/batch_size),
                    validation_data=val_images,validation_steps=ceil(val.shape[0]/32),
                    epochs=epochs,callbacks=[check,lrsched])

In [ ]:
# load the model back
model_v4 = load_model('/content/gdrive/My Drive/aptos2019-blindness-detection/model_v4',compile=False)
model_v4.summary()

In [ ]:
# generate the predictions on the test set
predictions = model_v4.predict_generator(test_images,steps=np.ceil(test.shape[0]/32))

In [ ]:
true_labels = test['diagnosis'].to_numpy()
pre1=predictions*4
for i in range(pre1.shape[0]):
    if(pre1[i]<=0.5):
        pre1[i]=0
    elif(pre1[i]>0.5 and pre1[i]<=1.5):
        pre1[i]=1
    elif(pre1[i]>1.5 and pre1[i]<=2.5):
        pre1[i]=2
    elif(pre1[i]>2.5 and pre1[i]<=3.5):
        pre1[i]=3
    else:
        pre1[i]=4
pred_labels = pre1

In [ ]:
# compute performance metrics
weighted_f1 = round(f1_score(true_labels, pred_labels, average = 'weighted'), 4) # 0.3874
accuracy = round(accuracy_score(true_labels, pred_labels), 4) # 0.3753
kappa = round(cohen_kappa_score(true_labels, pred_labels), 4) # 0.0183

# display the metrics
print("Accuracy: {}".format(accuracy) + '\n'
      + "Weigjted F1: {}".format(weighted_f1) + '\n'
      + "Cohen's Kappa: {}".format(kappa))

# display the confusion matrix
confusion_matrix(true_labels, pred_labels)

## Transfer Learning

In [ ]:
# import the file of labels
labels = pd.read_csv('data1/train.csv')
# add file type to the column of file names
labels['id_code'] = labels['id_code'].astype(str) + '.png'
# encode the label column to a categorical variable
labels['diagnosis'] = labels['diagnosis']

In [ ]:
# import a well-pretrained model with the tops layers unfrozen
input_shape=(384,384,3)
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)
x_model = base_model.output
x_model = GlobalAveragePooling2D(name='globalaveragepooling2d')(x_model)
inception_model = Model(inputs=base_model.input, outputs=x_model)

In [ ]:
# perform feature extraction to all the images using the pretrained model
features=np.zeros((labels.shape[0],2048))
batch_size=64
def load_image( path, img_size):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (img_size, img_size))
    return img/255.0
count=0
for start in range(0, len(labels), batch_size):
    end = min(start + batch_size, len(labels))
    df_temp=labels.iloc[start:end]
    X=np.zeros((df_temp.shape[0],384,384,3))
    for j in range(df_temp.shape[0]):
        img=load_image(os.path.join('data1/train_images',df_temp.iloc[j,0]),384)
        X[j]=img
    pre=inception_model.predict(X)
    features[start:end]=pre

In [ ]:
# put the extracted features into the dataframe containing the images paths
for i in range(2048):
    labels['f_'+str(i)]=0
labels.loc[:,'f_0':]=features

In [ ]:
# randomly split the labeled images into a training set and a test set in a ratio of 8:2
train_and_val, test = train_test_split(labels, test_size=0.2,random_state=1002965829)
# randomly subset 1/10 of the training set to form a validation set
train, val = train_test_split(train_and_val, test_size=0.1,random_state=1002965829)

In [ ]:
K.clear_session()
# load the encoder model
encoder_model=load_model('/content/gdrive/My Drive/aptos2019-blindness-detection/encoder_model')

### CNN V5 (pretraining + transfer learning + classification)

In [ ]:
# the first input comes from the enocder model
x_model = encoder_model.output
x_model = Flatten()(x_model)

x_model = Dense(64, activation='relu')(x_model)
x_model = Dense(64, activation='relu')(x_model)

# the second input comes from the transfer learning
inputs1=Input(shape=(2048,))#迁移模型输出
x= Dense(64, activation='relu')(inputs1)
x_all=concatenate([x_model,x])
x_all=Dense(64, activation='relu')(x_all)
predictions = Dense(5, activation='softmax', name='output_layer')(x_all)
model_v5 = Model(inputs=[encoder_model.input,inputs1], outputs=predictions)

In [ ]:
filepath =  '/content/gdrive/My Drive/aptos2019-blindness-detection/model_v5'

check=ModelCheckpoint(filepath=filepath,monitor='val_acc',save_best_only=True,save_weights_only=False,mode='max')
lrsched = OneCycleScheduler(lr_max=1e-3, n_data_points=train.shape[0],
        epochs=epochs, batch_size=batch_size, verbose=0)

In [ ]:
model_v5.compile(loss=categorical_crossentropy(),
              optimizer=Adam(lr=1e-3,decay=3e-4), metrics=['acc'])
model_v5.fit_generator(train_imgs,steps_per_epoch=ceil(train.shape[0]/batch_size),validation_data=val_imgs,validation_steps=ceil(val.shape[0]/32),epochs=epochs,callbacks=[check,lrsched])

In [ ]:
# generate predicted labels for the test set
model_v5 = load_model('/content/gdrive/My Drive/aptos2019-blindness-detection/model_v5',compile=False)
model_v5.summary()

In [ ]:
# convert the outputs to certain categories
pred_labels = predictions.argmax(axis = 1).astype(str)
true_labels = test['diagnosis'].to_numpy()

# compute performance metrics
weighted_f1 = round(f1_score(true_labels, pred_labels, average = 'weighted'), 4)
accuracy = round(accuracy_score(true_labels, pred_labels), 4)
kappa = round(cohen_kappa_score(true_labels, pred_labels), 4)

# display the metrics
print("Accuracy: {}".format(accuracy) + '\n'
      + "Weigjted F1: {}".format(weighted_f1) + '\n'
      + "Cohen's Kappa: {}".format(kappa))

# display the confusion matrix
confusion_matrix(true_labels, pred_labels)

### CNN V6 (pretraining + transfer learning + regression)

We combine the features that are extracted respectively from the autoencoder and the transfer learning, and use them to train a classification/regression network

In [ ]:
# the first input comes from the enocder model
x_model = encoder_model.output
x_model = Flatten()(x_model)

x_model = Dense(64, activation='relu')(x_model)
x_model = Dense(64, activation='relu')(x_model)

# the second input comes from the transfer learning
inputs1=Input(shape=(2048,))#迁移模型输出
x= Dense(64, activation='relu')(inputs1)
x_all=concatenate([x_model,x])
x_all=Dense(64, activation='relu')(x_all)
predictions = Dense(1, activation='linear', name='output_layer')(x_all)
model_v6 = Model(inputs=[encoder_model.input,inputs1], outputs=predictions)

In [ ]:
filepath =  '/content/gdrive/My Drive/aptos2019-blindness-detection/model_v6'

check=ModelCheckpoint(filepath=filepath,monitor='val_mae',save_best_only=True,save_weights_only=False,mode='min')
lrsched = OneCycleScheduler(lr_max=1e-3, n_data_points=train.shape[0],
        epochs=epochs, batch_size=batch_size, verbose=0)

In [ ]:
model_v6.compile(loss=smooth_L1_loss,
              optimizer=Adam(lr=1e-3,decay=3e-4), metrics=['mae'])
model_v6.fit_generator(train_imgs,steps_per_epoch=ceil(train.shape[0]/batch_size),validation_data=val_imgs,validation_steps=ceil(val.shape[0]/32),epochs=epochs,callbacks=[check,lrsched])

In [ ]:
# generate predicted labels for the test set
model_v6 = load_model('/content/gdrive/My Drive/aptos2019-blindness-detection/model_v6',compile=False)
model_v6.summary()

In [ ]:
predictions = model_v6.predict_generator(test_imgs,steps=np.ceil(test.shape[0]/32))

In [ ]:
true_labels = test['diagnosis'].to_numpy()
pre1=predictions*4
for i in range(pre1.shape[0]):
    if(pre1[i]<=0.5):
        pre1[i]=0
    elif(pre1[i]>0.5 and pre1[i]<=1.5):
        pre1[i]=1
    elif(pre1[i]>1.5 and pre1[i]<=2.5):
        pre1[i]=2
    elif(pre1[i]>2.5 and pre1[i]<=3.5):
        pre1[i]=3
    else:
        pre1[i]=4
pred_labels = pre1

In [ ]:
# compute performance metrics
weighted_f1 = round(f1_score(true_labels, pred_labels, average = 'weighted'), 4) # 0.3874
accuracy = round(accuracy_score(true_labels, pred_labels), 4) # 0.3753
kappa = round(cohen_kappa_score(true_labels, pred_labels), 4) # 0.0183

# display the metrics
print("Accuracy: {}".format(accuracy) + '\n'
      + "Weigjted F1: {}".format(weighted_f1) + '\n'
      + "Cohen's Kappa: {}".format(kappa))

# display the confusion matrix
confusion_matrix(true_labels, pred_labels)